In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import db_helper
import datetime
import networkx as nx
from datetime import datetime
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import importlib
import graph_constructor as GC
import n2v as N2V
import utils
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import sys
import random

In [234]:
import gc
gc.collect()

183844

In [4]:
PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'
MONTH_TAG_EN="month"
YEAR_TAG_EN="year"
DAY_TAG_EN="day"

MONTH_NAMES_EN = {1:"JAN",2:"FEB",3:"MAR",4:"APR",
          5:"MAY",6:"JUN",7:"JUL",8:"AUG",
         9:"SEP",10:"OCT",11:"NOV",12:"DEC"}

MONTH_TAG_TR="AY"
YEAR_TAG_TR="YIL"
DAY_TAG_TR="GUN"

MONTH_NAMES_TR = {1:"OCA",2:"SUB",3:"MAR",4:"NIS",
          5:"MAY",6:"HAZ",7:"TEM",8:"AGU",
         9:"EYL",10:"EKI",11:"KAS",12:"ARA"}

#### If you don't have json data, read data from db and then convert them into json to easy load-save. Reading data from db is not useful.

In [272]:
db = db_helper.DBHelper()
db.connectMysql(user="root",passwd="Hasan5695*",db="SocialMediaDB")
tweets = db.getTweetsFromMysql("TweetsShort")
print(tweets[0])
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)

############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 



In [270]:
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)

In [135]:
# USE this if you have json data
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')
allFriendships = db_helper.getAllFriendsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/friends.json')

In [271]:
print('TotalTweet#:',len(tweets))
print(tweets[0],tweets[len(tweets)-1])

print('\n\nExample AllFriendShips:\n',allFriendships[:2])
userFriends = [('u100341775',x['FriendID'], {'weight':1}) for x in allFriendships[:] if x['UserID']=='u100341775']
print('Friends of an example user:\n',userFriends[:5])

TotalTweet#: 93753
############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 
 ############ Tweet: ############ 
SendByScreenName:ahmet_iyimaya 
UserID:u3228821009 
TweetID:t851752159603568640 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2017-04-11 14:01:54 
Text:Yenimahalle'de İlçe teşkilatımız ile birliktetyiz. #Referandum çalışmalarımız tüm hızıyla devam ediyor. #Anayasa https://t.co/FOTZWbki2e 



Example AllFriendShips:
 [{'UserID': 'u100341775', 'FriendID': 'u1014141614'}, {'UserID': 'u100341775', 'FriendID': 'u120840897'}]
Friends of an example user:
 [('u100341775', 'u1014141614', {'weight': 1}), ('u100341775', 'u120840897', {'weight': 1}), ('u100341775', 'u1494034783', {'weight': 1}), ('u100341775', 'u154236788', {'weight': 1}), ('u100341775', 'u155534196', {'weight': 1})]

### Load NER Model

In [233]:
%%time
sys.path.append(PROJECT_PATH + '/src/tagger')

from loader import prepare_sentence
from utilsTagger import create_input, iobes_iob, iob_ranges, zero_digits
from model import Model

modelPath = PROJECT_PATH+'/dataset/NER_MODEL/tr2'
print('NER_ModelPath:',modelPath)

nerModel = Model(model_path=modelPath)
parameters = nerModel.parameters

# Load reverse mappings
word_to_id, char_to_id, tag_to_id = [
    {v: k for k, v in x.items()}
    for x in [nerModel.id_to_word, nerModel.id_to_char, nerModel.id_to_tag]
]

# Load the model
_, f_eval = nerModel.build(training=False, **parameters)
nerModel.reload()

print('Model Parameters:',parameters)

def extract_ner(line):
    if line:
        # Ahmet'in Turkiye'ye gidisi icin-> [Ahmet,in,Turkiye,ye,gidisi,icin]
        words_ini = line.rstrip().replace("'"," ").split()
        
        # Lowercase sentence
        if parameters['lower']:
            line = line.lower()
        # Replace all digits with zeros
        if parameters['zeros']:
            line = zero_digits(line)
        words = line.rstrip().replace("'"," ").split()
        # Prepare input
        sentence = prepare_sentence(words, word_to_id, char_to_id,
                                    lower=parameters['lower'])
        input = create_input(sentence, parameters, False)
        # Decoding
        if parameters['crf']:
            y_preds = np.array(f_eval(*input))[1:-1]
        else:
            y_preds = f_eval(*input).argmax(axis=1)
            
        y_preds = [nerModel.id_to_tag[y_pred] for y_pred in y_preds]
        # Output tags in the IOB2 format
        if parameters['tag_scheme'] == 'iobes':
            y_preds = iobes_iob(y_preds)
        
        #print('word_ini:', words_ini)
        #print('preds:',y_preds)
        return words_ini, y_preds
    return None

NER_ModelPath: /home/hmenn/Workspace/CSE496_Node2Vec_2018/dataset/NER_MODEL/tr2
Compiling...
Model Parameters: OrderedDict([('tag_scheme', 'iobes'), ('lower', False), ('zeros', False), ('char_dim', 25), ('char_lstm_dim', 25), ('char_bidirect', True), ('word_dim', 300), ('word_lstm_dim', 100), ('word_bidirect', True), ('pre_emb', 'dataset/wiki.tr.vec'), ('all_emb', False), ('cap_dim', 0), ('crf', True), ('dropout', 0.5), ('lr_method', 'adam')])
CPU times: user 7.81 s, sys: 803 ms, total: 8.61 s
Wall time: 8.75 s


In [255]:
def create_all_graph(tweets, allFriendships):
    G1 = nx.Graph()
    G1.add_node(YEAR_TAG_TR)
    G1.add_node(MONTH_TAG_TR)
    G1.add_node(DAY_TAG_TR)

    prevYear = None
    prevMonth = None
    prevDay = None

    prevYear = tweets[0].date.year
    prevMonth = tweets[0].date.month
    prevDay = tweets[0].date.day
        
    for i,tweet in enumerate(tweets):
        if (i+1) %1000 == 0:
            print(i,' th tweet was added.')
                
        ################# ADD DATE NODES #################
        G1.add_edge(YEAR_TAG_TR,str(tweet.date.year),weight=1.0)
        if(prevYear < tweet.date.year):
            G1.add_edge(str(prevYear),str(tweet.date.year),weight=1.0)
            prevYear = tweet.date.year
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
          
        monthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[tweet.date.month],year=tweet.date.year)
        G1.add_edge(str(tweet.date.year),monthFormat,weight=1.0)
        
        if(prevMonth < tweet.date.month):
            prevMonthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[prevMonth],year=tweet.date.year)
            G1.add_edge(prevMonthFormat,monthFormat,weight=1.0)
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
        
        #G1.add_edge(MONTH_TAG,monthFormat,weight=1.0)
        fullDateFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
        if prevDay < tweet.date.day:
            prevFullDateFormat = "{day}.{month}.{year}".format(day=prevDay,month=tweet.date.month,year=tweet.date.year)
            G1.add_edge(prevFullDateFormat,fullDateFormat,weight=1.0)
            prevDay = tweet.date.day
        G1.add_edge(monthFormat,fullDateFormat,weight=1.0)
        
        ################# ADD TWEET INFORMATIONS LIKE TEXT, USER #################
        # date -> tweet
        G1.add_edge(fullDateFormat,tweet.tweetID,weight=1.0)
        
                
        # if user will be added first time, add friend connections
        if not G1.has_node(tweet.userID):
            friends = [x['FriendID'] for x in allFriendships if x['UserID']==tweet.userID]
            # now check if friends tweeted before
            userFriends = []
            for friend  in friends:
                for t in tweets:
                    if t.userID==friend:
                        userFriends.append((tweet.userID,friend, {'weight':1.0}))
            if len(userFriends) != 0:
                print("User:", tweet.userID," - FriendsLen:", len(userFriends))            
            # add friendship connections
            G1.add_edges_from(userFriends)
            
        # user -> tweet connections were established
        G1.add_edge(tweet.userID, tweet.tweetID,weight=1.0)
                 
        # if tweet replied and db has actual(replied tweet, add to graph)
        # cevap verilen tweet db de ise, o tweete baglanti ekle
        for i in tweets:
            if i.tweetID == tweet.inReplyToStatusID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID,weight=1.0)
                #print('Replied:',tweet.tweetID, '<->',tweet.inReplyToStatusID)
                
        ################# ADD NER's #################
        
        words, ners = extract_ner(tweet.text)
        for w,n in zip(words, ners):
            if n != 'O':
                w = w.lower()
                print(w,n)
                G1.add_edge(tweet.tweetID, w,weight=1.0)
        print('######')
        
        
    return G1

In [253]:
dataset = tweets[0:100]
dataset += tweets[3000:3050]
dataset += tweets[5000:5100]
dataset += tweets[8000:8050]
len(dataset)

300

In [256]:
%%time
graph = create_all_graph(dataset, allFriendships)
print("Number of edges: ",graph.number_of_edges())
print("Number of nodes: ",graph.number_of_nodes())

User: u1623755515  - FriendsLen: 40
######
faiz B-MISC
######
i̇stanbul B-LOC
######
User: u547625189  - FriendsLen: 1
faiz B-MISC
######
######
User: u1594078644  - FriendsLen: 37
######
######
westbrook B-PER
######
basketbol B-MISC
litvanya B-LOC
######
basketbol B-MISC
######
######
basketbol B-MISC
fransa B-LOC
######
######
basketbol B-MISC
takımı B-ORG
i̇spanya B-LOC
######
######
######
erkek B-MISC
basketbol B-MISC
######
oyuncular B-MISC
######
2016/17 B-MISC
######
User: u565848357  - FriendsLen: 77
######
######
######
######
mete B-PER
######
######
######
######
######
######
######
köprü B-MISC
######
######
######
######
######
######
######
######
######
abbas B-PER
######
######
abd/nato B-PER
######
av. B-PER
osman I-PER
karakuş, I-PER
aziz I-PER
yıldırım I-PER
kumpas B-PER
######
######
######
######
######
######
######
######
######
######
oyuncular B-MISC
######
cahil B-PER
######
######
######
######
aziz B-MISC
######
######
tarih B-MISC
######
avrupa B-MISC
##

In [ ]:
# save and read edge list
#nx.write_edgelist(graph, path=PROJECT_PATH+"/outputs/grid.edgelist", delimiter=":")
#graph = nx.read_edgelist(path="grid.edgelist", delimiter=":")

In [258]:
%%time
graphN2V = N2V.Graph(graph, is_directed = False, p=1.0, q=1.0)
graphN2V.preprocess_transition_probs()
walks = graphN2V.simulate_walks(1000, 80)
model = N2V.learn_embeddings(walks,"graphAll")

Walk iteration:
1000 / 1000
CPU times: user 42min 19s, sys: 3.02 s, total: 42min 22s
Wall time: 16min 35s


In [259]:
graph['YIL'] #,graph[]
#model.wv.most_similar('t764606955562360832')

AtlasView({'2016': {'weight': 1.0}, '2017': {'weight': 1.0}})

In [ ]:
utils.draw_tsne(model, PROJECT_PATH + '/outputs/allTSNE.pdf')

In [ ]:
plt.figure(1, figsize=(50,50))
nx.draw_kamada_kawai(graph,node_size=5000,font_size=50,with_labels=True, width=3, edge_color="b", arrows=True, arrowsize=30)
plt.savefig(PROJECT_PATH+'/outputs/graphAll.pdf')
plt.show()

### Cosine Similarities for Times

In [273]:
graph['u1623755515']

AtlasView({'u1278512364': {'weight': 1.0}, 'u1499266621': {'weight': 1.0}, 'u1829708546': {'weight': 1.0}, 'u18426124': {'weight': 1.0}, 'u23186079': {'weight': 1.0}, 'u2424673684': {'weight': 1.0}, 'u4327240396': {'weight': 1.0}, 'u4856670141': {'weight': 1.0}, 'u583863506': {'weight': 1.0}, 'u61171559': {'weight': 1.0}, 'u732590069681393664': {'weight': 1.0}, 'u768542424': {'weight': 1.0}, 't763141405766250496': {'weight': 1.0}, 'u565848357': {'weight': 1.0}, 'u471859539': {'weight': 1.0}, 't819588562698010624': {'weight': 1.0}})

In [260]:
column_names = ['2016', 'AGU.2016', '10.8.2016', "16.8.2016", 'EKI.2016','29.10.2016','30.10.2016','2017','OCA.2017', '12.1.2017']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,2016,AGU.2016,10.8.2016,16.8.2016,EKI.2016,29.10.2016,30.10.2016,2017,OCA.2017,12.1.2017
2016,1.000000,0.642372,0.492105,0.153328,0.742586,0.347860,0.208554,0.766375,0.569516,0.181680
AGU.2016,0.642372,1.000000,0.753962,0.335239,0.587421,0.292050,0.230589,0.427036,0.319298,0.145191
10.8.2016,0.492105,0.753962,1.000000,0.275777,0.464674,0.205210,0.197475,0.354981,0.344252,0.261786
16.8.2016,0.153328,0.335239,0.275777,1.000000,0.136094,0.137177,0.195119,0.102880,0.107122,0.181643
EKI.2016,0.742586,0.587421,0.464674,0.136094,1.000000,0.575391,0.449526,0.484274,0.336304,0.125472
29.10.2016,0.347860,0.292050,0.205210,0.137177,0.575391,1.000000,0.398337,0.230422,0.192817,0.193661
30.10.2016,0.208554,0.230589,0.197475,0.195119,0.449526,0.398337,1.000000,0.150760,0.185048,0.224175
2017,0.766375,0.427036,0.354981,0.102880,0.484274,0.230422,0.150760,1.000000,0.835190,0.381484
OCA.2017,0.569516,0.319298,0.344252,0.107122,0.336304,0.192817,0.185048,0.835190,1.000000,0.631471
12.1.2017,0.181680,0.145191,0.261786,0.181643,0.125472,0.193661,0.224175,0.381484,0.631471,1.000000


### Cosine Similarities for Users

In [248]:
column_names = ['u1623755515','u565848357','u700397072063795200','u296406287']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,u1623755515,u565848357,u700397072063795200,u296406287
u1623755515,1.000000,0.175785,0.245315,0.189162
u565848357,0.175785,1.000000,0.172820,0.346993
u700397072063795200,0.245315,0.172820,1.000000,0.083847
u296406287,0.189162,0.346993,0.083847,1.000000


### Cosine Similarities for Tweets

In [263]:
column_names = ['t765296806276268032','t765338545871400961', 't765562268767678464','t763141405766250496','basketbol']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,t765296806276268032,t765338545871400961,t765562268767678464,t763141405766250496,basketbol
t765296806276268032,1.000000,0.473438,0.523164,0.132089,0.334164
t765338545871400961,0.473438,1.000000,0.410450,0.264506,0.365649
t765562268767678464,0.523164,0.410450,1.000000,0.168840,0.235586
t763141405766250496,0.132089,0.264506,0.168840,1.000000,0.244844
basketbol,0.334164,0.365649,0.235586,0.244844,1.000000


In [243]:
%%time
words, preds = extract_ner("Hasan Türkiye'ye taşındı.")
print(words, preds)

['Hasan', 'Türkiye', 'ye', 'taşındı.'] ['B-PER', 'B-LOC', 'O', 'O']
CPU times: user 76.8 ms, sys: 93.6 ms, total: 170 ms
Wall time: 161 ms


t765466344707989504, t765582802343657472 ve t765648654883512321 aynı gun atılmıstır ama ılk ıkısı extra olarak basketboldan konusmuslardır.

In [268]:
column_names = ['t765562268767678464','t763141405766250496','basketbol','t765466344707989504','t765582802343657472','t765648654883512321']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,t765562268767678464,t763141405766250496,basketbol,t765466344707989504,t765582802343657472,t765648654883512321
t765562268767678464,1.000000,0.168840,0.235586,0.336817,0.355017,0.443149
t763141405766250496,0.168840,1.000000,0.244844,0.123604,0.303882,0.155905
basketbol,0.235586,0.244844,1.000000,0.644623,0.664798,0.359757
t765466344707989504,0.336817,0.123604,0.644623,1.000000,0.627713,0.477728
t765582802343657472,0.355017,0.303882,0.664798,0.627713,1.000000,0.434969
t765648654883512321,0.443149,0.155905,0.359757,0.477728,0.434969,1.000000
